In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from src.modulos.dataset import Uti
from src.modulos.rede import ModeloNeural

from torch import optim, nn
from torch.utils.data import DataLoader
import torch


from sklearn.metrics import accuracy_score

import numpy as np

import pandas as pd

import time

In [3]:
args = dict(
    batch_size=10,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    weight_decay = 1e-3,
    lr = 1e-4,
    epoch = 100,
)

In [4]:
train_set = Uti('../dados/x_train.csv', '../dados/y_train.csv')
test_set = Uti('../dados/x_test.csv', '../dados/y_test.csv')

train_loader = DataLoader(train_set, args['batch_size'])
test_loader = DataLoader(test_set, args['batch_size'])

In [5]:
input_size = 228
hidden_size = 456
second_hidden = 912
out_size = 2

net = ModeloNeural(input_size, hidden_size, second_hidden, out_size)
net = net.to(args['device'])
net

ModeloNeural(
  (features): Sequential(
    (0): Linear(in_features=228, out_features=456, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=456, out_features=912, bias=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=912, out_features=2, bias=True)
  )
)

In [6]:
otimizador = optim.Adam(net.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
perda = nn.CrossEntropyLoss().to(args['device'])

In [7]:
def train(net, train_loader, epoch : int):
    start = time.time() # Marcador de tempo
    net.train()

    perda_epoca = []
    pred_list, rotulo_list = [], []
    for batch in train_loader:

        # Input
        dado, rotulo = batch
        
        #Cast para GPU
        dado = dado.to(args['device'])
        rotulo = rotulo.to(args['device']).squeeze()

        # zerar parametros gradientes
        otimizador.zero_grad()

        # forward + 
        saida = net(dado)
        loss = perda(saida, rotulo)
        

        # Salvando dados
        perda_epoca.append(loss.cpu().data)         # Loss
        _, predict = torch.max(saida, 1)            # Predicao
        pred_list.append(predict.cpu().numpy())     
        rotulo_list.append(rotulo.cpu().numpy())    # rotulos

        
        # backward + optmize
        loss.backward()
        otimizador.step()
    

      
    pred_list = np.concatenate(pred_list)
    rotulo_list = np.concatenate(rotulo_list)

    acuracia = accuracy_score(rotulo_list, pred_list) * 100
    perda_epoca = np.array(perda_epoca)

    end = time.time()  

    print(f'Epoch {epoch:>4} - train_score {acuracia:>6.2f} - loss {perda_epoca.mean():0.2f} - time {end-start:0.2f}', end=' // ')

    
    

In [8]:
def valida(net, test_loader):
    start = time.time()
    net.eval()
    
    perda_epoca = []
    pred_list, rotulo_list = [], []

    with torch.no_grad():
        for batch in test_loader:
            dado, rotulo = batch
            
            # Cast para GPU
            dado = dado.to(args['device'])
            rotulo = rotulo.to(args['device']).squeeze()

            # Forward
            predicao = net(dado)
            loss = perda(predicao, rotulo)

            # Salvando dados
            perda_epoca.append(loss.cpu().data)
            _, predict = torch.max(predicao, 1)             
            pred_list.append(predict.cpu().numpy())     
            rotulo_list.append(rotulo.cpu().numpy())

     
    pred_list = np.concatenate(pred_list)
    rotulo_list = np.concatenate(rotulo_list)

    acuracia = accuracy_score(rotulo_list, pred_list) * 100
    perda_epoca = np.array(perda_epoca)

    end = time.time()   

    print(f'val_score {acuracia:>6.2f} - val_loss {perda_epoca.mean():0.2f} - time {end-start:0.2f}') 
        




In [9]:
for i in range(1, 500):
    train(net, train_loader, i)
    valida(net, test_loader)

.65 - val_loss 0.31 - time 0.04
Epoch  308 - train_score  96.74 - loss 0.09 - time 0.26 // val_score  91.23 - val_loss 0.32 - time 0.04
Epoch  309 - train_score  96.85 - loss 0.09 - time 0.30 // val_score  91.94 - val_loss 0.33 - time 0.04
Epoch  310 - train_score  96.64 - loss 0.09 - time 0.33 // val_score  90.28 - val_loss 0.34 - time 0.03
Epoch  311 - train_score  97.46 - loss 0.08 - time 0.25 // val_score  91.00 - val_loss 0.34 - time 0.04
Epoch  312 - train_score  96.24 - loss 0.10 - time 0.29 // val_score  92.18 - val_loss 0.32 - time 0.04
Epoch  313 - train_score  96.24 - loss 0.09 - time 0.33 // val_score  92.89 - val_loss 0.32 - time 0.04
Epoch  314 - train_score  95.73 - loss 0.11 - time 0.28 // val_score  92.18 - val_loss 0.32 - time 0.04
Epoch  315 - train_score  96.54 - loss 0.08 - time 0.29 // val_score  93.60 - val_loss 0.31 - time 0.04
Epoch  316 - train_score  97.05 - loss 0.08 - time 0.34 // val_score  90.28 - val_loss 0.35 - time 0.03
Epoch  317 - train_score  96.85 

tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0')

In [10]:
pd.read_csv('../dados/x_train.csv').shape

(983, 228)

In [11]:
pd.read_csv('../dados/x_test.csv').shape

(422, 228)